# --- BƯỚC 0: CÀI ĐẶT THƯ VIỆN ---
# Cài đặt thư viện peft (cho LoRA) và các thư viện khác
# -q (quiet) để ẩn bớt output
print("--- BƯỚC 0: Đang cài đặt thư viện (khoảng 1-2 phút) ---")
!pip install transformers[torch] datasets evaluate accelerate scikit-learn peft -q
print("Cài đặt hoàn tất!")


# --- BƯỚC 1: IMPORT VÀ CÀI ĐẶT CHUNG ---
import transformers
import datasets
import evaluate
import numpy as np
import pandas as pd # Dùng để in bảng so sánh
import time
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import f1_score, accuracy_score

# Imports cho LoRA
from peft import get_peft_model, LoraConfig, TaskType

print("\n--- BƯỚC 1: Tải Dataset và Tokenizer (Chung) ---")
# Tải dataset UIT-VSFC từ HuggingFace
dataset = load_dataset("uitnlp/vietnamese_students_feedback")

# Tải Tokenizer của PhoBERT
model_name = "vinai/phobert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
print("Tải dataset và tokenizer thành công!")


# --- BƯỚC 2: TIỀN XỬ LÝ VÀ METRICS (Chung) ---
print("\n--- BƯỚC 2: Đang tiền xử lý dữ liệu và định nghĩa Metrics ---")

# Hàm tiền xử lý (giống project đơn giản)
def preprocess_function(examples):
    examples["label"] = examples["sentiment"]
    return tokenizer(examples["sentence"], truncation=True, padding="max_length", max_length=128)

tokenized_datasets = dataset.map(preprocess_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(["sentence", "sentiment", "topic"])

# Hàm tính toán metrics (giống project đơn giản)
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1 = f1_score(labels, predictions, average="macro")
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc, "f1_macro": f1}

print("Tiền xử lý và định nghĩa metrics hoàn tất!")


# --- BƯỚC 3: CHẠY BASELINE (FULL FINE-TUNING) ---
print("\n" + "="*50)
print(" BẮT ĐẦU RUN 1: FULL FINE-TUNING (BASELINE)")
print("="*50)
print("Đây là bước tốn thời gian nhất (khoảng 15 phút)...")

# Tải model PhoBERT gốc (cho lần chạy 1)
model_full = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=3
)

# Định nghĩa tham số huấn luyện
training_args_full = TrainingArguments(
    output_dir="phobert-full-finetune", # Thư mục lưu model 1
    num_train_epochs=3,                 # 3 epochs là đủ để so sánh
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    logging_steps=100,
)

trainer_full = Trainer(
    model=model_full,
    args=training_args_full,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Bắt đầu đo thời gian và huấn luyện
start_time_full = time.time()
trainer_full.train()
end_time_full = time.time()
time_full = end_time_full - start_time_full

print("\nĐánh giá Baseline trên tập Test...")
results_full = trainer_full.evaluate(tokenized_datasets["test"])

# Lưu lại kết quả
f1_full = results_full['eval_f1_macro']
params_full = sum(p.numel() for p in model_full.parameters() if p.requires_grad)

print(f"RUN 1 HOÀN TẤT: F1 = {f1_full:.4f}, Time = {time_full:.2f}s, Params = {params_full:,}")


# --- BƯỚC 4: CHẠY VỚI LoRA (PEFT) ---
print("\n" + "="*50)
print(" BẮT ĐẦU RUN 2: LoRA (PEFT) FINE-TUNING")
print("="*50)
print("Bước này sẽ nhanh hơn đáng kể (khoảng 5-10 phút)...")

# TẢI LẠI model gốc (RẤT QUAN TRỌNG, để train từ đầu)
model_lora = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=3
)

# === 💡 ĐÂY LÀ PHẦN CODE "ĂN ĐIỂM" CỦA BẠN ===
# 1. Định nghĩa LoRA Config
peft_config = LoraConfig(
    task_type=TaskType.SEQUENCE_CLASSIFICATION, # Tác vụ
    r=8,                  # Rank (thường là 8, 16, 32)
    lora_alpha=16,        # Alpha (thường gấp đôi r)
    lora_dropout=0.1,
    target_modules=["query", "key", "value"] # Áp dụng LoRA vào các lớp Q, K, V
)

# 2. "Bọc" model gốc bằng LoRA
model_lora = get_peft_model(model_lora, peft_config)

# 3. In ra số tham số có thể huấn luyện (WOW FACTOR)
print("--- SO SÁNH THAM SỐ HUẤN LUYỆN ---")
model_lora.print_trainable_parameters()
# ============================================

# Dùng chung tham số training, chỉ đổi thư mục output
training_args_lora = TrainingArguments(
    output_dir="phobert-lora-finetune", # Thư mục lưu model 2
    num_train_epochs=3,
    learning_rate=2e-5, # Giữ nguyên LR để so sánh công bằng
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    logging_steps=100,
)

trainer_lora = Trainer(
    model=model_lora, # Dùng model đã bọc LoRA
    args=training_args_lora,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Bắt đầu đo thời gian và huấn luyện
start_time_lora = time.time()
trainer_lora.train()
end_time_lora = time.time()
time_lora = end_time_lora - start_time_lora

print("\nĐánh giá LoRA trên tập Test...")
results_lora = trainer_lora.evaluate(tokenized_datasets["test"])

# Lưu lại kết quả
f1_lora = results_lora['eval_f1_macro']
params_lora = sum(p.numel() for p in model_lora.parameters() if p.requires_grad)

print(f"RUN 2 HOÀN TẤT: F1 = {f1_lora:.4f}, Time = {time_lora:.2f}s, Params = {params_lora:,}")


# --- BƯỚC 5: TỔNG KẾT VÀ SO SÁNH ---
print("\n" + "="*50)
print(" BƯỚC 5: SO SÁNH KẾT QUẢ CUỐI CÙNG")
print("="*50)

# Tạo bảng so sánh
data = {
    "Phương pháp": ["1. Full Fine-Tuning (Baseline)", "2. LoRA (PEFT)"],
    "Test F1-macro": [f1_full, f1_lora],
    "Thời gian train (giây)": [time_full, time_lora],
    "Số tham số train": [f"{params_full:,}", f"{params_lora:,}"],
    "Tỉ lệ tham số": [1.0, params_lora / params_full]
}
df = pd.DataFrame(data)

# In bảng so sánh dạng Markdown (đẹp)
print(df.to_markdown(index=False, floatfmt=(".4f", ".4f", ".2f", "s", ".5%")))

print("\n--- NHẬN XÉT TỰ ĐỘNG ---")
print(f"✅ Hiệu suất: LoRA đạt {f1_lora/f1_full:.2%} hiệu suất F1-score so với Full Fine-Tuning.")
print(f"✅ Hiệu quả: LoRA chỉ huấn luyện {params_lora/params_full:.2%} số tham số.")
print(f"✅ Tốc độ: Thời gian huấn luyện của LoRA nhanh hơn {1-(time_lora/time_full):.1%}.")

import transformers
import datasets
import evaluate
import numpy as np
import pandas as pd # Dùng để in bảng so sánh
import time
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import f1_score, accuracy_score

# Imports cho LoRA
from peft import get_peft_model, LoraConfig, TaskType

print("\n--- BƯỚC 1: Tải Dataset và Tokenizer (Chung) ---")
# Tải dataset UIT-VSFC từ HuggingFace
dataset = load_dataset("uitnlp/vietnamese_students_feedback")

In [17]:
# --- BƯỚC 0: CÀI THƯ VIỆN ---
print("--- BƯỚC 0: Đang cài đặt thư viện (khoảng 1-2 phút) ---")
!pip install -q transformers[torch] datasets evaluate accelerate scikit-learn peft
print("Cài đặt hoàn tất!")

# --- BƯỚC 1: IMPORT ---
import os, time
import numpy as np
import pandas as pd
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import f1_score, accuracy_score
from peft import get_peft_model, LoraConfig, TaskType

print("\n--- BƯỚC 1: Kiểm tra và tải dataset ---")

# MODEL / TOKENIZER
model_name = "vinai/phobert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# --- (Toàn bộ phần code tải/chuẩn hóa dataset của bạn giữ nguyên) ---
# --- (Bắt đầu từ đây) ---
csv_train = "train.csv"
csv_test = "test.csv"
csv_val_candidates = ["val.csv", "validation.csv"]

def try_load_from_csv():
    files = {}
    if os.path.exists(csv_train):
        files["train"] = csv_train
    if os.path.exists(csv_test):
        files["test"] = csv_test
    for cand in csv_val_candidates:
        if os.path.exists(cand):
            files["validation"] = cand
            break
    if not files:
        return None
    print("-> Load dataset từ CSV local:", files)
    ds = load_dataset("csv", data_files=files)
    return ds

dataset = None
try:
    dataset = try_load_from_csv()
    if dataset is None:
        print("Không tìm thấy CSV local. Tiến hành clone repo dataset từ Hugging Face...")
        if not os.path.exists("vietnamese_students_feedback"):
            !git clone https://huggingface.co/datasets/uitnlp/vietnamese_students_feedback
        script_path = None
        for root, dirs, files in os.walk("vietnamese_students_feedback"):
            for f in files:
                if f.endswith(".py"):
                    script_path = os.path.join(root, f)
                    break
            if script_path:
                break
        if script_path is None:
            raise RuntimeError("Không tìm thấy file .py trong repo đã clone.")
        print("-> Load dataset từ script local:", script_path)
        dataset = load_dataset(script_path)
    print("Dataset load thành công. Các split:", dataset)
except Exception as e:
    print("LỖI khi load dataset:", e)
    raise

def find_text_col(columns):
    for cand in ["text", "sentence", "content", "comment"]:
        if cand in columns:
            return cand
    return columns[0]

def find_label_col(columns):
    for cand in ["label", "sentiment", "labels"]:
        if cand in columns:
            return cand
    return None

for split in dataset.keys():
    cols = dataset[split].column_names
    text_col = find_text_col(cols)
    label_col = find_label_col(cols) or (cols[1] if len(cols) > 1 else None)
    rename_map = {}
    if text_col and text_col != "text":
        rename_map[text_col] = "text"
    if label_col and label_col != "label":
        rename_map[label_col] = "label"
    if rename_map:
        dataset[split] = dataset[split].rename_columns(rename_map)

print("Sau rename, các cột train:", dataset["train"].column_names)

label_values = sorted(list(set(dataset["train"]["label"])))[:10]
print("Ví dụ các giá trị label (train):", label_values)

if isinstance(label_values[0], str):
    unique_labels = sorted(list(set(dataset["train"]["label"])))
    label2id = {lab: idx for idx, lab in enumerate(unique_labels)}
    id2label = {v: k for k, v in label2id.items()}
    print("Detected label strings. Mapping label2id:", label2id)
    def map_label(example):
        example["label"] = label2id[example["label"]]
        return example
    dataset = dataset.map(map_label)
else:
    unique_labels = sorted(list(set(dataset["train"]["label"])))
    label2id = {str(l): int(l) for l in unique_labels}
    id2label = {int(v): k for k, v in label2id.items()}

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

print("Đang token hóa (map)...")
tokenized_datasets = dataset.map(preprocess_function, batched=True)

for col in ["text", "sentence", "sentiment", "topic"]:
    if col in tokenized_datasets["train"].column_names and col != "label":
        try:
            tokenized_datasets = tokenized_datasets.remove_columns([col])
        except Exception:
            pass

print("Tokenize xong. Columns now:", tokenized_datasets["train"].column_names)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    f1 = f1_score(labels, preds, average="macro")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1_macro": f1}

import transformers
tf_ver = transformers.__version__
print("transformers version:", tf_ver)

common_training_args = dict(
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    logging_steps=100,
    fp16=True,
    report_to="none"
)
# --- (Hết phần chuẩn bị) ---


# --- RUN 1: Full fine-tuning baseline ---
print("\n" + "="*50)
print("RUN 1: FULL FINE-TUNING (BASELINE)")
print("="*50)

model_full = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(id2label),
    id2label=id2label,
    label2id={v:k for k,v in id2label.items()}
)

training_args_full = TrainingArguments(
    output_dir="phobert-full-finetune",
    learning_rate=2e-5,
    **common_training_args
)

trainer_full = Trainer(
    model=model_full,
    args=training_args_full,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets.get("validation", tokenized_datasets.get("val", None)) or tokenized_datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

start_time_full = time.time()
trainer_full.train()
time_full = time.time() - start_time_full

print("\nĐánh giá Baseline trên tập Test...")
results_full = trainer_full.evaluate(eval_dataset=tokenized_datasets["test"])
f1_full = results_full.get("eval_f1_macro", results_full.get("f1_macro", None))
params_full = sum(p.numel() for p in model_full.parameters() if p.requires_grad)
print(f"RUN 1 HOÀN TẤT: F1 = {f1_full}, Time = {time_full:.2f}s, Params = {params_full:,}")

# --- RUN 2: LoRA r=8 ---
print("\n" + "="*50)
print("RUN 2: LoRA (PEFT) r=8")
print("="*50)

model_lora = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(id2label))

peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["query", "key", "value"]
)
model_lora = get_peft_model(model_lora, peft_config)
model_lora.print_trainable_parameters()

training_args_lora = TrainingArguments(
    output_dir="phobert-lora-finetune",
    learning_rate=1e-4,
    **common_training_args
)

trainer_lora = Trainer(
    model=model_lora,
    args=training_args_lora,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets.get("validation", tokenized_datasets.get("val", None)) or tokenized_datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

start_time_lora = time.time()
trainer_lora.train()
time_lora = time.time() - start_time_lora

print("\nĐánh giá LoRA r=8 trên tập Test...")
results_lora = trainer_lora.evaluate(eval_dataset=tokenized_datasets["test"])
f1_lora = results_lora.get("eval_f1_macro", results_lora.get("f1_macro", None))
params_lora = sum(p.numel() for p in model_lora.parameters() if p.requires_grad)
print(f"RUN 2 HOÀN TẤT: F1 = {f1_lora}, Time = {time_lora:.2f}s, Params = {params_lora:,}")


# --- RUN 3: LoRA r=256 --- (✅ ĐÃ SỬA COMMENT)
print("\n" + "="*50)
print("RUN 3: LoRA (PEFT) r=256") # <-- ✅ Sửa ở đây
print("="*50)

model_lora_256 = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(id2label))
peft_config_256 = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=256,
    lora_alpha=512,
    lora_dropout=0.1,
    target_modules=["query", "key", "value"]
)
model_lora_256 = get_peft_model(model_lora_256, peft_config_256)
model_lora_256.print_trainable_parameters()

training_args_lora_256 = TrainingArguments(
    output_dir="phobert-lora-r256", # <-- Sửa tên thư mục (gợi ý)
    learning_rate=1e-4,
    **common_training_args
)

trainer_lora_256 = Trainer(
    model=model_lora_256,
    args=training_args_lora_256,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets.get("validation", tokenized_datasets.get("val", None)) or tokenized_datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

start_time_lora_256 = time.time()
trainer_lora_256.train()
time_lora_256 = time.time() - start_time_lora_256

print("\nĐánh giá LoRA r=256 trên tập Test...") # <-- ✅ Sửa ở đây
results_lora_256 = trainer_lora_256.evaluate(eval_dataset=tokenized_datasets["test"])
f1_lora_256 = results_lora_256.get("eval_f1_macro", results_lora_256.get("f1_macro", None))
params_lora_256 = sum(p.numel() for p in model_lora_256.parameters() if p.requires_grad)
print(f"RUN 3 HOÀN TẤT (r=256): F1 = {f1_lora_256}, Time = {time_lora_256:.2f}s, Params = {params_lora_256:,}") # <-- ✅ Sửa ở đây


# --- RUN 4: LoRA r=512 --- (✅ ĐÃ SỬA COMMENT)
print("\n" + "="*50)
print("RUN 4: LoRA (PEFT) r=512") # <-- ✅ Sửa ở đây
print("="*50)

model_lora_512 = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(id2label))
peft_config_512 = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=512,
    lora_alpha=1024,
    lora_dropout=0.1,
    target_modules=["query", "key", "value"]
)
model_lora_512 = get_peft_model(model_lora_512, peft_config_512)
model_lora_512.print_trainable_parameters()

training_args_lora_512 = TrainingArguments(
    output_dir="phobert-lora-r512", # <-- Sửa tên thư mục (gợi ý)
    learning_rate=1e-4,
    **common_training_args
)

trainer_lora_512 = Trainer(
    model=model_lora_512,
    args=training_args_lora_512,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets.get("validation", tokenized_datasets.get("val", None)) or tokenized_datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

start_time_lora_512 = time.time()
trainer_lora_512.train()
time_lora_512 = time.time() - start_time_lora_512

print("\nĐánh giá LoRA r=512 trên tập Test...") # <-- ✅ Sửa ở đây
results_lora_512 = trainer_lora_512.evaluate(eval_dataset=tokenized_datasets["test"])
f1_lora_512 = results_lora_512.get("eval_f1_macro", results_lora_512.get("f1_macro", None))
params_lora_512 = sum(p.numel() for p in model_lora_512.parameters() if p.requires_grad)
print(f"RUN 4 HOÀN TẤT (r=512): F1 = {f1_lora_512}, Time = {time_lora_512:.2f}s, Params = {params_lora_512:,}") # <-- ✅ Sửa ở đây


# --- So sánh kết quả ---
print("\n" + "="*50)
print("SO SÁNH KẾT QUẢ")
print("="*50)

df = pd.DataFrame({
    "Phương pháp": [
        "Full Fine-Tuning (Baseline)",
        "LoRA r=8",
        "LoRA r=256", # <-- ✅ Sửa ở đây
        "LoRA r=512"  # <-- ✅ Sửa ở đây
    ],
    "Test F1-macro": [f1_full, f1_lora, f1_lora_256, f1_lora_512],
    "Thời gian train (s)": [time_full, time_lora, time_lora_256, time_lora_512],
    "Số tham số train": [params_full, params_lora, params_lora_256, params_lora_512],
    "Tỉ lệ tham số vs Full": [
        1.0,
        params_lora / params_full if params_full else None,
        params_lora_256 / params_full if params_full else None,
        params_lora_512 / params_full if params_full else None
    ]
})
print(df.to_markdown(index=False))

print("\n--- NHẬN XÉT ---")
print(f"Baseline F1 = {f1_full}")
print(f"LoRA r=8 F1 = {f1_lora} ({params_lora/params_full:.2%} params of full)")
print(f"LoRA r=256 F1 = {f1_lora_256} ({params_lora_256/params_full:.2%} params of full)") # <-- ✅ Sửa ở đây
print(f"LoRA r=512 F1 = {f1_lora_512} ({params_lora_512/params_full:.2%} params of full)") # <-- ✅ Sửa ở đây

--- BƯỚC 0: Đang cài đặt thư viện (khoảng 1-2 phút) ---
Cài đặt hoàn tất!

--- BƯỚC 1: Kiểm tra và tải dataset ---
-> Load dataset từ CSV local: {'train': 'train.csv', 'test': 'test.csv', 'validation': 'val.csv'}
Dataset load thành công. Các split: DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 4080
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1050
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1020
    })
})
Sau rename, các cột train: ['text', 'label']
Ví dụ các giá trị label (train): ['NEG', 'NEU', 'POS']
Detected label strings. Mapping label2id: {'NEG': 0, 'NEU': 1, 'POS': 2}


Map:   0%|          | 0/4080 [00:00<?, ? examples/s]

Map:   0%|          | 0/1050 [00:00<?, ? examples/s]

Map:   0%|          | 0/1020 [00:00<?, ? examples/s]

Đang token hóa (map)...


...

trainable params: 28,904,451 || all params: 163,905,030 || trainable%: 17.6349


/tmp/ipython-input-2358062039.py:304: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_lora_32 = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro
1,0.948900,0.749173,0.675490,0.674515
2,0.701500,0.720902,0.692157,0.687673
3,0.635800,0.738043,0.695098,0.693573



Đánh giá LoRA r=32 trên tập Test...


RUN 4 HOÀN TẤT: F1 = 0.7063774024218145, Time = 133.33s, Params = 28,904,451

SO SÁNH KẾT QUẢ
| Phương pháp                 |   Test F1-macro |   Thời gian train (s) |   Số tham số train |   Tỉ lệ tham số vs Full |
|:----------------------------|----------------:|----------------------:|-------------------:|------------------------:|
| Full Fine-Tuning (Baseline) |        0.71242  |               479.522 |          135000579 |              1          |
| LoRA r=8                    |        0.657087 |                70.501 |            1035267 |              0.00766861 |
| LoRA r=16                   |        0.712679 |               104.58  |           14748675 |              0.109249   |
| LoRA r=32                   |        0.706377 |               133.332 |           28904451 |              0.214106   |

--- NHẬN XÉT ---
Baseline F1 = 0.7124195534083682
LoRA r=8 F1 = 0.6570871197580491 (0.77% params of full)
LoRA r=16 F1 = 0.712678616379205 (10.92% params of full)
LoRA r=32 F1 = 0